In [ ]:
# %%
# Importation des bibliothèques
import os
import json
import zipfile
import requests
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Configuration
plt.style.use('ggplot')
sns.set(style="whitegrid")
%matplotlib inline

print("=" * 70)
print("VÉRIFICATION DU GPU")
print("=" * 70)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Nombre de GPUs: {torch.cuda.device_count()}")
    print(f"GPU actuel: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    device = torch.device("cuda:0")
else:
    print("⚠️ CUDA non disponible, utilisation du CPU")
    device = torch.device("cpu")
print(f"Device utilisé: {device}")
print("=" * 70)

# %%
# Fonction pour configurer kaggle.json
def setup_kaggle_json():
    """Sélectionne le fichier kaggle.json si non présent"""
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_file = kaggle_dir / 'kaggle.json'
    kaggle_dir.mkdir(parents=True, exist_ok=True)

    if kaggle_file.exists():
        print(f"✅ kaggle.json déjà présent : {kaggle_file}")
    else:
        print("⚠️ kaggle.json non trouvé. Sélectionnez votre fichier Kaggle JSON.")
        try:
            import tkinter as tk
            from tkinter import filedialog
            root = tk.Tk()
            root.withdraw()
            file_path = filedialog.askopenfilename(
                title="Sélectionnez votre fichier kaggle.json",
                filetypes=[("JSON files", "*.json")]
            )
            if not file_path:
                raise FileNotFoundError("Aucun fichier sélectionné.")
            shutil.copy(file_path, kaggle_file)
            print(f"✅ kaggle.json copié vers {kaggle_file}")
        except ImportError:
            print("❌ tkinter non disponible. Copiez manuellement kaggle.json dans ~/.kaggle/")
            raise

    if os.name != 'nt':
        os.chmod(kaggle_file, 0o600)

# %%
# Fonction pour explorer la structure du dossier
def explore_directory(path, max_depth=3, current_depth=0):
    """Explore récursivement la structure du dossier"""
    path = Path(path)
    indent = "  " * current_depth

    if current_depth >= max_depth:
        return

    for item in sorted(path.iterdir())[:10]:  # Limite à 10 items par niveau
        if item.is_dir():
            count = len(list(item.glob('*.jpg'))) + len(list(item.glob('*.png')))
            print(f"{indent}📁 {item.name}/ {'(' + str(count) + ' images)' if count > 0 else ''}")
            explore_directory(item, max_depth, current_depth + 1)
        else:
            print(f"{indent}📄 {item.name}")

# %%
# Fonction pour télécharger ISIC 2020
def download_isic_2020(local_dir="./isic_dataset"):
    """Télécharge et extrait ISIC 2020 via l'API Kaggle"""
    data_path = Path(local_dir)
    data_path.mkdir(parents=True, exist_ok=True)

    try:
        from kaggle.api.kaggle_api_extended import KaggleApi
    except ImportError:
        print("❌ Kaggle API non installée. Installez-la avec : pip install kaggle")
        return None

    api = KaggleApi()
    api.authenticate()

    print("📥 Téléchargement et extraction directe du dataset ISIC 2020...")
    api.dataset_download_files(
        "andrewmvd/isic-2020",  # ← Dataset public
        path=str(data_path),
        unzip=True,
        quiet=False
    )

    print("\n🔍 Exploration de la structure du dataset :")
    explore_directory(data_path)

    # Recherche du dossier d'images
    possible_paths = [
        data_path / "ISIC_2020_Training_JPEG",
        data_path / "ISIC_2020_Training_JPEG",
        data_path / "train",
        data_path / "images",
    ]

    img_dir = None
    for path in possible_paths:
        if path.exists():
            jpg_count = len(list(path.glob('*.jpg')))
            if jpg_count > 0:
                img_dir = path
                break

    attr_file = data_path / "ISIC_2020_Training_GroundTruth.csv"

    if img_dir and img_dir.exists():
        total_images = len(list(img_dir.glob('*.jpg')))
        print(f"\n✅ Dataset prêt ! Nombre d'images : {total_images:,}")
        print(f"📂 Chemin des images : {img_dir}")
        print(f"📊 Fichier attributs : {attr_file if attr_file.exists() else '❌ Non trouvé'}")
        return str(img_dir), str(attr_file)
    else:
        print("\n⚠️ Impossible de localiser les images. Vérifiez la structure manuellement.")
        return None, None

# %%
# Exécution principale
setup_kaggle_json()
IMG_DIR, ATTR_FILE = download_isic_2020(local_dir="./isic_dataset")

if IMG_DIR and ATTR_FILE:
    print(f"\n🎉 Dataset ISIC 2020 téléchargé avec succès !")
    print(f"📁 Images : {IMG_DIR}")
    print(f"📊 Labels : {ATTR_FILE}")
else:
    print("\n❌ Échec du téléchargement. Vérifiez votre connexion et votre fichier kaggle.json.")
